# Why boost::phoenix?
## &#9658; Supports "Functional Programming"
## &#9658; Okay, but what is "Functional Programming"

# Why Functional Programming
## &#9658; Style of programming that encourages statelessness
## &#9658; Code produces the same output for a given input
## &#9658; "Referentially Transparent": Code can be replaced by its output
## &#9658; Output fully encapsulates the operations of the underlying code 

## &#9658; Primary means of writing Functional Code is the function
## &#9658; Writing stateless code improves overall stability and concurrency
## &#9658; Easier to debug
### For example:

# Imperative Style

In [1]:
// Define vector of scores, used throughout this presentation.
const std::vector<int> scores{4, 6, 3, 7, 10};

int sum = 0;
for (int score : scores) 
{
    sum += score;
}

std::cout << "average score: " << sum / (double)scores.size();

average score: 6

# Functional Style

In [2]:
std::cout << "average score: " << std::accumulate(scores.begin(), scores.end(), 0) / (double)scores.size();

average score: 6

# Back to boost::phoenix
## &#9658; Supports functional programming with function objects
## &#9658; Overloads the operator()
## &#9658; Has the attributes of objects, while still being callable

# Without boost::phoenix...

In [3]:
#include <vector>
#include <algorithm>
#include <iostream>

bool is_odd(int i) { return i % 2 == 1; }

// Count the number of odds in the scores using a simple function.
std::cout << std::count_if(scores.begin(), scores.end(), is_odd);

2

In [4]:
auto lambda = [](int i){ return i % 2 == 1; };

// Count the number of odds using a lambda expression.
std::cout << std::count_if(scores.begin(), scores.end(), lambda);

2

# With boost::phoenix...

In [5]:
#include <boost/phoenix/phoenix.hpp>
using namespace boost::phoenix::placeholders;
using namespace boost::phoenix;

// Define a phoenix function object that accepts one value and uses the modulo operator to determine 
// if the value is evenly divisible by two.
auto phoenix = arg1 % 2 == 1;
std::cout << std::count_if(scores.begin(), scores.end(), phoenix);

2

# Phoenix Functions
## &#9658; boost::phoenix::placeholders::arg1, consists only of a function body
## &#9658; &#9658; arg1 global instance of a unary function object
## &#9658; &#9658; Placeholder for the value passed
## &#9658; Other placeholders are available
## &#9658; &#9658; boost::phoenix::placeholders::arg2, boost::phoenix::placeholders::arg3

# Phoenix Functions vs. Lambdas
## &#9658; No data types required
### For example:

In [6]:
// Define a lambda function that calculates the total number of odd scores of type integer.
auto lambda_for_ints = [](int i){ return i % 2 == 1; };
std::cout << std::count_if(scores.begin(), scores.end(), lambda_for_ints);

2

In [7]:
const std::vector<long> long_scores{4, 6, 3, 7, 10};

// Same phoenix function defined above will work with this new long vector.
std::cout << std::count_if(long_scores.begin(), long_scores.end(), phoenix) << std::endl;

// And also works with the vector of integer scores.
std::cout << std::count_if(scores.begin(), scores.end(), phoenix) << std::endl;


2
2


# Deferred Execution of Phoenix Functions
## &#9658; val() returns a function object initialized with the value specified
## &#9658; Phoenix overloads operators for value types
## &#9658; Functions are not evaluated until they are referenced

In [8]:
using namespace boost::phoenix::placeholders;
using namespace boost::phoenix;

// Define a Phoenix function that combines multiple operators, with deferred execution.
auto phoenix_deferred = arg1 > val(2) && arg1 % val(2) == val(1);
std::cout << std::count_if(scores.begin(), scores.end(), phoenix) << std::endl;

2


In [9]:
// To demonstrate how arg1, arg2, and arg3 work, the specified entry in the list is returned.
std::cout << arg1(1, 2, 3, 4, 5) << std::endl;
std::cout << arg2(1, 2, 3, 4, 5) << std::endl;
std::cout << arg3(1, 2, 3, 4, 5) << std::endl;

1
2
3


In [10]:
using namespace boost::phoenix::placeholders;
using namespace boost::phoenix;

// Create function object with constant value.
auto v = val(2);

// When accessed as a function, the function object always returns the value provided in the initializer
std::cout << v() << std::endl;

2


# How to Create Your Own Phoenix Functions
## &#9658; Pass a function object to the template boost::phoenix::function
## &#9658; Must define a type called "result_type"
### For example:

In [11]:
// Define an object with an operator that determines if the value is odd.
// Also, define type "result_type" to allows Phoenix to know the return data type.
struct is_odd_impl
{
    typedef bool result_type;

    template <typename T>
        bool operator()(T t) const { return t % 2 == 1; }
};

// Declare the phoenix function for use later.
boost::phoenix::function<is_odd_impl> is_odd_from_object;

// Use the function to evaluate each score in the vector.
using namespace boost::phoenix::placeholders;
std::cout << std::count_if(scores.begin(), scores.end(), is_odd_from_object(arg1)) << std::endl;

2


## &#9658; Also possible to convert function into a Phoenix function
### For example:

In [12]:
// Define the function (not part of an object) that will be converted to a Phoenix function.
bool is_odd_function(int i) { return i % 2 == 1; }

In [13]:
#include <boost/phoenix/phoenix.hpp>

// Make the function into a Phoenix function: 
// parm1 - return type
// parm2 - name of the Phoenix function
// parm3 - name of the standard function
// parm4 - number of parameters
BOOST_PHOENIX_ADAPT_FUNCTION(bool, is_odd_phoenix, is_odd_function, 1);


input_line_22:7:1: error: namespaces can only be defined in global or namespace scope
BOOST_PHOENIX_ADAPT_FUNCTION(bool, is_odd_phoenix, is_odd_function, 1);
^
/usr/local/include/boost/phoenix/function/adapt_function.hpp:45:5: note: expanded from macro 'BOOST_PHOENIX_ADAPT_FUNCTION'
    {                                                                           \
    ^
input_line_22:7:1: error: expected expression
/usr/local/include/boost/phoenix/function/adapt_function.hpp:64:5: note: expanded from macro 'BOOST_PHOENIX_ADAPT_FUNCTION'
    template <BOOST_PHOENIX_typename_A(N)>                                      \
    ^


In [14]:
using namespace boost::phoenix;
using namespace boost::phoenix::placeholders;

// Use the Phoenix function created above:
std::cout << std::count_if(scores.begin(), scores.end(), is_odd_phoenix(arg1)) << std::endl;

input_line_23:5:58: error: use of undeclared identifier 'is_odd_phoenix'
std::cout << std::count_if(scores.begin(), scores.end(), is_odd_phoenix(arg1)) << std::endl;
                                                         ^


# Phoenix Functions Simulating C++ Keywords
## &#9658; If the condition is true, code in brackets is executed
### For example:

In [15]:
using namespace boost::phoenix;  
using namespace boost::phoenix::placeholders;

int count = 0;
std::for_each(scores.begin(), scores.end(), 
    // Define the function object using the Phoenix if_ object.
    if_(arg1 > 2 && arg1 % 2 == 1)
    [
      ++ref(count)
    ]);
std::cout << count << std::endl;

2


# References

Tutorial:
https://theboostcpplibraries.com/boost.phoenix
    
Docker Image with Boost::Phoenix support:
https://cloud.docker.com/u/jywilson/repository/docker/jywilson/xeus-cling
    
    
Referential Transparency:
https://www.sitepoint.com/what-is-referential-transparency/


